In [22]:
import os

import numpy as np

from keras.models import load_model

import cv2

import pandas as pd

In [8]:
model_path = 'models/model.h5'
test_img_path = '../data/test/'

In [21]:
for root, dirs, test_img_names in os.walk(test_img_path):
    test_img_ids = [img_name[:-4] for img_name in test_img_names]
    test_img_paths = [test_img_path + img_name for img_name in test_img_names]

In [14]:
model = load_model(model_path)

OSError: Unable to open file (unable to open file: name = 'models/model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [19]:
test_img_names

['0005bb9630.png',
 '000a68e46c.png',
 '000c8dfb2a.png',
 '000d0a5f6c.png',
 '001ef8fc87.png',
 '002124aa19.png',
 '002af5d1e8.png',
 '00323f1910.png',
 '00329dc15c.png',
 '0035c56490.png',
 '0041cb8c49.png',
 '0043a01a19.png',
 '00565e793d.png',
 '005855cd72.png',
 '005b02dd7c.png',
 '0066451ce3.png',
 '00690a4185.png',
 '006d9df08a.png',
 '00707c7864.png',
 '007176e42a.png',
 '007c7c7dcf.png',
 '00801127b0.png',
 '008315cbd8.png',
 '008f8c8aaa.png',
 '0091fb2dab.png',
 '0092c53387.png',
 '0094822667.png',
 '009bfb3a78.png',
 '009d3365bc.png',
 '009e9c5f22.png',
 '00a052d822.png',
 '00a5318192.png',
 '00a6bfc7a7.png',
 '00a738b887.png',
 '00a9b78f49.png',
 '00aa6a3958.png',
 '00aaa5b7f4.png',
 '00b3ad08c8.png',
 '00b68675b4.png',
 '00b6d3a31f.png',
 '00bbed5966.png',
 '00bf25f9f9.png',
 '00c06147b5.png',
 '00c153007b.png',
 '00c473f654.png',
 '00c6225718.png',
 '00c6536758.png',
 '00cbbf2293.png',
 '00cde1fc96.png',
 '00cdefa5d6.png',
 '00d26ea627.png',
 '00d51ae232.png',
 '00e68bfb1b

In [18]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
#     print(runs, runs[1::2])
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [17]:
model_input_size = 101 if '101' in model_path else '128'

In [13]:
rle_masks = []
for img_path in test_img_paths:
    img = cv2.imread(img_path, 0)
    need_resize = model_input_size != 101
    if need_resize:
        img = cv2.resize(img, (model_input_size , model_input_size), interpolation=cv2.INTER_CUBIC)
    img = img.reshape((1,model_input_size, model_input_size, 1))
    prediction = model.predict(img)
    prediction = (prediction > 0.7).astype(float)
    if need_resize:
        prediction = cv2.resize(prediction, (101,101), interpolation=cv2.INTER_AREA)
    prediction = prediction >= 0.99
    rle_masks.append(rle_encode(prediction))
    

In [2]:
submission = {'id':test_img_ids, 'rle_mask':rle_masks}

submission_df = pd.DataFrame.from_dict(submission)
# save to csv